# 决策树判断鱼类和非鱼类

---
Author: Neo
Date: 2022年2月3日
---

In [3]:
import numpy as np
import pandas as pd
from math import log
import operator

In [6]:
def calcshannonent(dataset):
    numentries = len(dataset)
    labelcounts = {}
    for featvec in dataset:
        currentlabel = featvec[-1]
        if currentlabel not in labelcounts.keys():
            labelcounts[currentlabel] = 0
        labelcounts[currentlabel] += 1
    shannonent = 0.0
    for key in labelcounts:
        prob = float(labelcounts[key]) / numentries
        shannonent -= prob * log(prob, 2)
    return shannonent

In [11]:
def majortycnt(classlist):
    classcount = {}
    for vote in classlist:
        if vote not in classcount.keys():
            classcount[vote] = 0
        classcount[vote] += 1
    sortedclasscount = sorted(classcount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedclasscount[0][0]

In [ ]:
def splitdataset(dataset, index, value):
    retdataset = []
    for featvec in dataset:
        if featvec[index] == value:
            reducedfeatvec = featvec[:index]
            reducedfeatvec.extend(featvec[index + 1:])
            retdataset.append(reducedfeatvec)
    return retdataset

In [8]:
def choosebestfeaturetosplit(dataset):
    numfeatures = len(dataset[0]) - 1
    baseentropy = calcshannonent(dataset)
    bestinfogain, bestfeature = 0.0, -1
    for i in range(numfeatures):
        featlist = [a[i] for a in dataset]
        uniquevals = set(featlist)
        newentropy = 0.0
        for value in uniquevals:
            subdataset =

In [10]:
def createtree(dataset, labels):
    classlist = [a[-1] for a in dataset]
    if classlist.count(classlist[0]) == len(classlist):
        return classlist[0]
    if len(dataset[0]) == 1:
        return majortycnt(classlist)